In [8]:
!pip install gradio -q

from openai import OpenAI
from pydantic import BaseModel
import gradio as gr

# Initialize OpenAI client
client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key="Your_API_Key")

class Sentiment(BaseModel):
    sentiment: str
    responsible_area: str

def provide_sentiment(feedback: str):
    try:
        completion = client.chat.completions.create(
            temperature=0,
            model="google/learnlm-1.5-pro-experimental:free",
            messages=[
                {"role": "system", "content": """You are an assistant that analyzes hotel guest feedback and provides sentiment analysis in JSON format. The sentiment should be Positive, Negative, or Neutral. If the sentiment is Negative, identify the specific area responsible for the feedback, including one or more of the following categories: 'Room Quality', 'Cleanliness', 'Staff Service', 'Food & Beverage', 'Amenities', 'Check-in/Check-out Process', 'Location', 'Pricing', 'Noise', 'Safety', 'Guest Room Comfort', 'Bathroom Facilities', 'Hotel Ambience', 'Parking', 'Eco-friendliness', 'Pet Policy', 'Business Facilities', 'Spa & Wellness', and 'Other'. If no specific area applies, return 'N/A'. Ensure that responses are clear and helpful."""},
                {"role": "user", "content": "Here is a feedback for analysis: \"The hotel room was fantastic, and the staff was friendly!\""},
                {"role": "assistant", "content": "{\"sentiment\": \"Positive\", \"responsible_area\": \"N/A\"}"},
                {"role": "user", "content": "Here is a feedback for analysis: \"The room was dirty, and the staff were rude.\""},
                {"role": "assistant", "content": "{\"sentiment\": \"Negative\", \"responsible_area\": \"Room Quality, Staff Service\"}"},
                {"role": "user", "content": "Here is a feedback for analysis: \"The breakfast was great, but the Wi-Fi was very slow.\""},
                {"role": "assistant", "content": "{\"sentiment\": \"Neutral\", \"responsible_area\": \"N/A\"}"},
                {"role": "user", "content": f"Here is a feedback for analysis: \"{feedback}\""}
            ]
        )


        if completion.choices and completion.choices[0].message:
            sentiment_response = completion.choices[0].message.content
            return sentiment_response
        else:
            return "{\"sentiment\": \"Error\", \"responsible_area\": \"N/A\"}"  # Default error response if fields are missing

    except Exception as e:
        print(f"Error occurred: {e}")
        return "{\"sentiment\": \"Error\", \"responsible_area\": \"N/A\"}"  # Default error response in case of an exception


def analyze_feedback_ui(feedback):
    """Function to integrate with Gradio."""
    result = provide_sentiment(feedback)
    return result


interface = gr.Interface(
    fn=analyze_feedback_ui,
    inputs=gr.Textbox(
        lines=5,
        placeholder="Enter hotel guest feedback here...",
        label="Hotel Feedback",
    ),
    outputs=gr.JSON(label="Sentiment Analysis Result"),
    title="🏨  Hotel Feedback Sentiment Analyzer",
    description=(
        "🌟 Analyze hotel guest feedback for sentiment (Positive, Negative, or Neutral) "
        "and identify areas of concern if the sentiment is negative. 🚀 Powered by AI!"
    ),
    examples=[
        ["The hotel room was fantastic, and the staff was friendly!"],
        ["The room was dirty, and the staff were rude."],
        ["The breakfast was great, but the Wi-Fi was very slow."],
    ],
    theme="compact",
    allow_flagging="never",
    live=True,
)


interface.launch(share=True)


/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:1049: UserWarning: Cannot load compact. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/compact (Request ID: Root=1-6777f9ec-123e96c1688a1d135ded9457;20a34462-91dc-4354-a045-856ee286c6e0)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")
/usr/local/lib/python3.10/dist-packages/gradio/interface.py:399: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://72ee40d3b5d7cb2905.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from openai import OpenAI
import openai
from pydantic import BaseModel
import pandas as pd

# Initialize OpenAI client
client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key="YOUR_API_KEY")

# Define the sentiment response model
class Sentiment(BaseModel):
    sentiment: str
    responsible_area: str

# Function to analyze sentiment with detailed feedback analysis
def provide_sentiment(feedback: str):
    try:
        completion = client.chat.completions.create(
            temperature=0,
            model="google/learnlm-1.5-pro-experimental:free",
            messages=[
                {"role": "system", "content": """You are an assistant that analyzes hotel guest feedback and provides sentiment analysis in JSON format. The sentiment should be Positive, Negative, or Neutral. If the sentiment is Negative, identify the specific area responsible for the feedback, including one or more of the following categories: 'Room Quality', 'Cleanliness', 'Staff Service', 'Food & Beverage', 'Amenities', 'Check-in/Check-out Process', 'Location', 'Pricing', 'Noise', 'Safety', 'Guest Room Comfort', 'Bathroom Facilities', 'Hotel Ambience', 'Parking', 'Eco-friendliness', 'Pet Policy', 'Business Facilities', 'Spa & Wellness', and 'Other'. If no specific area applies, return 'N/A'. Ensure that responses are clear and helpful."""},
                {"role": "user", "content": f"Here is a feedback for analysis: \"{feedback}\""}
            ]
        )

        # Validate and extract the response
        if completion.choices and completion.choices[0].message:
            sentiment_response = completion.choices[0].message.content
            return sentiment_response
        else:
            return "{\"sentiment\": \"Error\", \"responsible_area\": \"N/A\"}"  # Return a default error response if the expected fields are missing

    except Exception as e:
        print(f"Error occurred: {e}")
        return "{\"sentiment\": \"Error\", \"responsible_area\": \"N/A\"}"  # Return a default error response in case of an exception

# Load the dataset (you can replace this with the actual dataset you are using, e.g., 'feedback.csv')
def analyze_feedback_from_file(file_path: str, feedback_column: str):
    # Load dataset using pandas (CSV or Excel file)
    try:
        # Read dataset (CSV example, you can change this to `pd.read_excel()` if it's an Excel file)
        df = pd.read_csv(file_path)

        # Add new columns to store sentiment analysis results
        df['Sentiment'] = None
        df['Responsible Area'] = None

        # Loop through each feedback entry in the dataset and analyze it
        for index, row in df.iterrows():
            feedback = row[feedback_column]  # Use the provided column name for feedback
            sentiment_analysis = provide_sentiment(feedback)

            # Parse sentiment analysis result from JSON-like string to extract sentiment and responsible area
            if sentiment_analysis.startswith("{") and sentiment_analysis.endswith("}"):
                sentiment_data = eval(sentiment_analysis)  # Safely convert the string to a dictionary
                df.at[index, 'Sentiment'] = sentiment_data.get('sentiment')
                df.at[index, 'Responsible Area'] = sentiment_data.get('responsible_area')
            else:
                df.at[index, 'Sentiment'] = 'Error'
                df.at[index, 'Responsible Area'] = 'N/A'

        # Save the updated dataset to a new CSV file or Excel file
        output_file = "feedback_analysis_output.csv"
        df.to_csv(output_file, index=False)
        print(f"Analysis complete. Results saved to {output_file}")

    except Exception as e:
        print(f"Error occurred while processing the file: {e}")

if __name__ == "__main__":
    # Specify the path to your local dataset (CSV or Excel)
    file_path = input("Enter the path to the feedback dataset (e.g., feedback.csv): ")
    feedback_column = input("Enter the column name containing the feedback (e.g., Review): ")

    # Analyze feedback data from the file
    analyze_feedback_from_file(file_path, feedback_column)


Enter the path to the feedback dataset (e.g., feedback.csv): /content/updated_customer_feedback.csv
Enter the column name containing the feedback (e.g., Review): Reviews
